In [41]:
import pandas as pd
import glob
import csv
from collections import Counter

def convert_excel_to_csv(input_file, output_file):
    df = pd.read_excel(input_file, engine='openpyxl')  
    df.to_csv(output_file, index=False, encoding='utf-8') 

convert_excel_to_csv("logs.txt.xlsx", "logs.csv")
                
        

In [42]:
def split_csv_data(input_file, num_of_rows=100000, name_of_split_file="split_file_"):
    chunk_counter = 1

    for chunk in pd.read_csv(input_file, chunksize=num_of_rows, encoding='utf-8'):
        chunk.to_csv(f"{name_of_split_file}{chunk_counter}.csv", index=False, encoding="utf-8")
        chunk_counter += 1

    print(f"הקובץ פוצל בהצלחה ל-{chunk_counter-1} חלקים!")

split_csv_data("logs.csv", 100000, "split")



הקובץ פוצל בהצלחה ל-10 חלקים!


In [48]:
#פונקציה המחשבת שכיחויות לכל חלק בנפרד.

def count_errors_in_chunk(chunk_file):
    error_counter = Counter()

    with open(chunk_file, 'r', encoding='utf-8') as file:
        for line in file:
            error_code = line.strip().split()[4]
            error_code = error_code.replace('"',"")
            error_counter[error_code] += 1

    output_file = f"{chunk_file}_errors.csv"

    with open(f"{chunk_file}_errors.csv", 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["Error_Code", "Count"])
        writer.writerows(error_counter.items())

    print(f"שמירת התוצאה הסתיימה עבור {chunk_file} -> {output_file}")
    print(f"common {error_counter.most_common(1)}")

count_errors_in_chunk('split1.csv')
count_errors_in_chunk('split2.csv')


שמירת התוצאה הסתיימה עבור split1.csv -> split1.csv_errors.csv
common [('ERR_404', 20207)]
שמירת התוצאה הסתיימה עבור split2.csv -> split2.csv_errors.csv
common [('ERR_404', 20110)]


In [81]:
df=pd.read_csv("split1.csv_errors.csv")
df

,Error_Code,Count
0,ERR_400,19992
1,ERR_500,19816
2,INFO_200,19927
3,WARN_101,20059
4,ERR_404,20207


In [82]:
#הכנסה למערך את כל המצביעים לקבצים המחולקים
errors_csv = glob.glob("split*.csv_errors.csv") 
errors_csv

['split1.csv_errors.csv', 'split2.csv_errors.csv']

In [84]:
errors = ['ERR_400', 'ERR_500', 'WARN_101', 'ERR_404']
new_df = pd.DataFrame(columns=errors)

new_df

,ERR_400,ERR_500,WARN_101,ERR_404


In [101]:
def build_file_of_error_with_freq(arr_of_error_files, dest_file):
    if dest_file.empty:
        dest_file = pd.DataFrame([[0] * len(dest_file.columns)], columns=dest_file.columns)
    for file_name in arr_of_error_files:
        try:
            with open(file_name, 'r', newline='', encoding='utf-8') as file:
                for line in file:
                    error_code = line.split(',')[0]
                    count = line.split(',')[1]
                    if error_code in dest_file.columns:
                        dest_file.loc[0,error_code] += int(count)
                    
        except FileNotFoundError:
            print(f"שגיאה: הקובץ {file_name} לא נמצא.")
        except OSError as e:
            print(f"שגיאת מערכת בקובץ {file_name}: {e}")
    return dest_file


updated_df = build_file_of_error_with_freq(errors_csv, new_df)

updated_df.to_csv("all_errors_with_frequency.csv", index=False, encoding='utf-8')
df= pd.read_csv("all_errors_with_frequency.csv")
df

,ERR_400,ERR_500,WARN_101,ERR_404
0,39698,39834,40139,40317


In [1]:
cnt = Counter()
for error in updated_df.columns():
    cnt[error] += updated_df.loc[0, error]

NameError: name 'Counter' is not defined

In [104]:
updated_df.columns()

TypeError: 'Index' object is not callable